<a href="https://colab.research.google.com/github/KurniaKhaikal/Cifar10k_TransferLearning/blob/main/TransferLearning_Cifar10_NewModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy
import numpy as np
import os
import json
import cv2
import sklearn.metrics as metric
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"khaikal","key":"9e492a7553b8e713debda6f4f6810151"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d oxcdcd/cifar10

100% 175M/175M [00:01<00:00, 153MB/s]
100% 175M/175M [00:01<00:00, 161MB/s]


In [ ]:
import os
import zipfile

local_zip = '/content/cifar10.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders


splitfolders.ratio("/content/cifar10/test", output="/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2",
    seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

Copying files: 10000 files [01:49, 91.09 files/s]


In [3]:
def euclidean(a, b):
	return np.linalg.norm(a - b)

In [4]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

In [5]:
base_dataset = "/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/"
class_dir = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
IMAGE_SIZE = (224, 224)

In [6]:
# indexing file images
dataset_test = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'val', class_item)
    for file in os.listdir(cur_dir):
        dataset_test.append(os.path.join(cur_dir, file))

In [7]:
print("len to retrieving:", len(dataset_test))

len to retrieving: 1000


In [8]:
dataset_test

['/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/428_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/667_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/7774_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/5043_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/4748_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/8476_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/422_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/3971_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/val/airplane/7103_airplane.png',
 '/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Ba

In [9]:
print("[INFO] load images Corel-Image 1k dataset...")
#  load images
val_images = []
for image_path in dataset_test:
    if ".png" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        val_images.append(image)

[INFO] load images Corel-Image 1k dataset...


In [11]:
val_images

[array([[[235, 201, 181],
         [235, 201, 181],
         [235, 201, 181],
         ...,
         [177, 171, 148],
         [177, 171, 148],
         [177, 171, 148]],
 
        [[235, 201, 181],
         [235, 201, 181],
         [235, 201, 181],
         ...,
         [177, 171, 148],
         [177, 171, 148],
         [177, 171, 148]],
 
        [[235, 201, 181],
         [235, 201, 181],
         [235, 201, 181],
         ...,
         [177, 171, 148],
         [177, 171, 148],
         [177, 171, 148]],
 
        ...,
 
        [[ 55, 106,  94],
         [ 55, 106,  94],
         [ 55, 106,  94],
         ...,
         [ 73, 132, 120],
         [ 73, 132, 120],
         [ 73, 132, 120]],
 
        [[ 55, 106,  94],
         [ 55, 106,  94],
         [ 55, 106,  94],
         ...,
         [ 73, 132, 120],
         [ 73, 132, 120],
         [ 73, 132, 120]],
 
        [[ 55, 106,  94],
         [ 55, 106,  94],
         [ 55, 106,  94],
         ...,
         [ 73, 132, 120],
  

In [10]:
print("[INFO] normalization...")
val_x = np.array(val_images).astype("float32") / 255.0

[INFO] normalization...


In [12]:
import tensorflow as tf
from tensorflow import keras

In [13]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=100, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(2,2), strides=(2,2), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(2,2), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(9216,input_shape=(12544,), activation='relu'),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(2, activation='sigmoid')
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 100)     2800      
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 100)    400       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 100)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 256)       102656    
                                                                 
 batch_normalization_1 (Batc  (None, 56, 56, 256)      1024      
 hNormalization)                                                 
                                                        

In [15]:
model.compile(
    optimizer=tf.optimizers.Adam(lr=0.000001),
    loss='binary_crossentropy',
    metrics=['accuracy','Recall','Precision']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
features = model.predict(val_x)

In [17]:
with open('/content/drive/MyDrive/Feature Extraction/NT_feature.json') as f:
  training_indexed = json.load(f)

In [42]:
val_x.shape

(1000, 224, 224, 3)

In [49]:
query_indexes = list(range(0, val_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features[i]
    results = perform_search(queryFeatures, training_indexed, max_results=10)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset_test[i].split("/")[11]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyError: ignored

In [ ]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))

recall values:
airplane 0.42999999999999994
automobile 0.598
bird 0.3400000000000001
cat 0.25000000000000006
deer 0.34600000000000003
dog 0.29000000000000004
frog 0.508
horse 0.3960000000000001
ship 0.512
truck 0.596
combined recall 0.4266

precision values:
airplane 0.84
automobile 0.93
bird 0.75
cat 0.7
deer 0.74
dog 0.67
frog 0.86
horse 0.75
ship 0.86
truck 0.91
combined precision 0.8009999999999999
